In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [16]:
data = pd.read_csv("jester-data-1.csv",header = None)
d = data.to_latex()
text_file = open("Output.txt", "w")
text_file.write(d)
text_file.close()

In [19]:
n_features = 2

user_ratings = data.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))
length = len(data)
label_length = len(data) * 0.10

In [20]:

#print(latent_item_features)
#print()
#print(latent_user_preferences)

       0      1      2      3      4     5      6     7     8      9    ...    \
0       74  -7.82   8.79  -9.66  -8.16 -7.52  -8.50 -9.85  4.17  -8.98  ...     
1      100   4.08  -0.29   6.36   4.37 -2.38  -9.66 -0.73 -5.34   8.88  ...     
2       49  99.00  99.00  99.00  99.00  9.03   9.27  9.03  9.27  99.00  ...     
3       48  99.00   8.35  99.00  99.00  1.80   8.16 -2.82  6.21  99.00  ...     
4       91   8.50   4.61  -4.17  -5.39  1.36   1.60  7.04  4.61  -0.44  ...     
5      100  -6.17  -3.54   0.44  -8.50 -7.09  -4.32 -8.69 -0.87  -6.65  ...     
6       47  99.00  99.00  99.00  99.00  8.59  -9.85  7.72  8.79  99.00  ...     
7      100   6.84   3.16   9.17  -6.21 -8.16  -1.70  9.27  1.41  -5.19  ...     
8      100  -3.79  -3.54  -9.42  -6.89 -8.74  -0.29 -5.29 -8.93  -7.86  ...     
9       72   3.01   5.15   5.15   3.01  6.41   5.15  8.93  2.52   3.01  ...     
10      36  -2.91   4.08  99.00  99.00 -5.73  99.00  2.48 -5.29  99.00  ...     
11     100   1.31   1.80   2

In [13]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not np.isnan(rating)):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            print(mse)                    